Data Loading

In [2]:
import pandas as pd
import os
import calendar

In [3]:
citybike_2021_df = pd.read_csv('citybike_2021_data.csv')

citybike_2021_df.head()

C:\Users\ianro\AppData\Local\Temp\ipykernel_26116\1942337203.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  citybike_2021_df = pd.read_csv('citybike_2021_data.csv')


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BBA33D73DECE976F,docked_bike,2021-02-26 16:38:54,2021-02-26 16:44:37,E 84 St & Park Ave,7243.04,E 78 St & 2 Ave,7057.07,40.778626,-73.957720,40.772797,-73.955778,casual
1,B63D7AFF9AC5B6D4,docked_bike,2021-02-17 11:09:11,2021-02-17 11:26:47,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.689622,-73.983043,member
2,52B829195C469D99,docked_bike,2021-02-26 18:33:29,2021-02-26 19:05:41,Macon St & Nostrand Ave,4214.03,Lefferts Pl & Franklin Ave,4222.02,40.680983,-73.950047,40.680342,-73.955769,casual
3,19C84ECA2B468476,docked_bike,2021-02-26 12:48:35,2021-02-26 13:07:24,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.689622,-73.983043,member
4,C0DDB771E70D9DF5,docked_bike,2021-02-25 17:23:22,2021-02-25 17:28:20,Madison Ave & E 26 St,6131.12,W 37 St & 5 Ave,6398.06,40.742685,-73.986713,40.750380,-73.983390,member


In [4]:
from datetime import datetime

# Convert 'ended_at' column to datetime
citybike_2021_df['ended_at'] = pd.to_datetime(citybike_2021_df['ended_at'])

# Filter data based on 'ended_at' column
citybike_2021_df = citybike_2021_df[citybike_2021_df['ended_at'] < datetime(2021, 6, 1)]

Average Length of Time

In [5]:
citybike_2021_df['started_at'] = pd.to_datetime(citybike_2021_df['started_at'])
citybike_2021_df['ended_at'] = pd.to_datetime(citybike_2021_df['ended_at'])

# Calculate the duration of the trip
citybike_2021_df['trip_duration'] = citybike_2021_df['ended_at'] - citybike_2021_df['started_at']

# Convert the duration to seconds for easier analysis
citybike_2021_df['trip_duration_seconds'] = citybike_2021_df['trip_duration'].dt.total_seconds()

citybike_2021_df['trip_duration_minutes'] = citybike_2021_df['trip_duration_seconds'] / 60

Average Distance (Haversine)

In [6]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of Earth in kilometers
    return distance

# Calculate Haversine distance for each row in the DataFrame and append it as a new column
citybike_2021_df['haversine_distance'] = citybike_2021_df.apply(lambda row: haversine_distance(row['start_lat'], row['start_lng'], row['end_lat'], row['end_lng']), axis=1)


Average Approximate Speed

In [7]:
# Convert kilometers to miles (1 kilometer = 0.621371 miles)
citybike_2021_df['haversine_distance_miles'] = citybike_2021_df['haversine_distance'] * 0.621371

# Calculate average approximate speed (in miles per hour)
citybike_2021_df['average_speed_mph'] = citybike_2021_df['haversine_distance_miles'] / (citybike_2021_df['trip_duration_minutes'] / 60)

# Display the DataFrame with the new column
print(citybike_2021_df[['trip_duration_minutes', 'haversine_distance', 'average_speed_mph']].head())


   trip_duration_minutes  haversine_distance  average_speed_mph
0               5.716667            0.668508           4.359795
1              17.600000            2.943338           6.234903
2              32.200000            0.487733           0.564714
3              18.816667            2.943338           5.831761
4               4.966667            0.900273           6.757898


In [8]:
import pandas as pd

# Convert 'started_at' and 'ended_at' columns to datetime
citybike_2021_df['started_at'] = pd.to_datetime(citybike_2021_df['started_at'])
citybike_2021_df['ended_at'] = pd.to_datetime(citybike_2021_df['ended_at'])

# Extract day of the week and hour for 'started_at'
citybike_2021_df['started_dayofweek'] = citybike_2021_df['started_at'].dt.dayofweek
citybike_2021_df['started_hour'] = citybike_2021_df['started_at'].dt.hour

# Extract day of the week and hour for 'ended_at'
citybike_2021_df['ended_dayofweek'] = citybike_2021_df['ended_at'].dt.dayofweek
citybike_2021_df['ended_hour'] = citybike_2021_df['ended_at'].dt.hour

# Display the DataFrame with the new columns
print(citybike_2021_df[['started_at', 'started_dayofweek', 'started_hour', 'ended_at', 'ended_dayofweek', 'ended_hour']].head())


           started_at  started_dayofweek  started_hour            ended_at  \
0 2021-02-26 16:38:54                  4            16 2021-02-26 16:44:37   
1 2021-02-17 11:09:11                  2            11 2021-02-17 11:26:47   
2 2021-02-26 18:33:29                  4            18 2021-02-26 19:05:41   
3 2021-02-26 12:48:35                  4            12 2021-02-26 13:07:24   
4 2021-02-25 17:23:22                  3            17 2021-02-25 17:28:20   

   ended_dayofweek  ended_hour  
0                4          16  
1                2          11  
2                4          19  
3                4          13  
4                3          17  


Export for Tableau Analysis

In [9]:
# Export DataFrame to CSV file
citybike_2021_df.to_csv('citybike_2021_data_customcalcs.csv', index=False)